## Análisis Precipitación Total Parque Nacional Terepaima

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

Directorio de trabajo

In [1]:
import os

print('> Directorio actual: ', os.getcwd())  
os.chdir('../')
print('> Directorio actual: ', os.getcwd()) 

> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics/terepaima
> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics


In [2]:
from utils.MONGO import CONEXION
from utils.UTILS import *
from datetime import datetime
import pandas as pd

from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Creando Coenxión con Mongo DB

In [4]:
# Creando la conexión con MongoDB
db = CONEXION.conexion()
db.list_collection_names()

['estimateMeteorological', 'meteorological', 'estimateSSTNino34', 'SSTNino34']

# Descargando la Información Precipitación

In [5]:
# Parque
park = 'terepaima'
id_point = 1

y_output = 'precip_t'
exogena = 'oni'

prediction_order = 12 # rango de prediccion
auto_order = 20*12 # componente autoregresiva
exog_order =  10*12# componente exogena qm
exog_delay = 1# componente exogena dm

f_activation = 'sigmoid'

# Parametros de modelos
patience = 15
epochs=500

Directorio del experimento

In [6]:
DIR = f'./{park}/'
experimento = f'experiments/narx/precipitacion/id_point_{id_point}'

try:
    os.mkdir(f'{DIR}{experimento}')
except:
    pass

Consulta de la data

In [7]:
# Realizando consulta
meteorological = db.meteorological.find({"park":park, 'id_point':id_point})

# Generando pandas dataframe
data_pandas = pd.DataFrame([file for file in meteorological])
data_pandas['periodo'] = data_pandas.time.apply(lambda x: datetime.fromordinal(x))
data_pandas['mes_year'] =  data_pandas['periodo'].dt.strftime('%B-%Y')
data_pandas.index = pd.to_datetime(data_pandas.periodo)
#data_pandas.head()

In [8]:
data_pandas

,_id,id_point,park,time,elevacion_maxima,elevacion_media,elevacion_mediana,latitud,longitud,ndvi_maxima,ndvi_media,ndvi_mediana,precipitacion_mm,time_actualizacion,periodo,mes_year
periodo,,,,,,,,,,,,,,,,
1970-01-01,63548350343c9f2921dbb11e,1,terepaima,719163,1632.0,1001.651794,986.0,9.96,-69.38,None,None,None,1.698088,738450,1970-01-01,January-1970
1970-02-01,63548350343c9f2921dbb12a,1,terepaima,719194,1632.0,1001.651794,986.0,9.96,-69.38,None,None,None,0.460838,738450,1970-02-01,February-1970
1970-03-01,63548350343c9f2921dbb147,1,terepaima,719222,1632.0,1001.651794,986.0,9.96,-69.38,None,None,None,0.559042,738450,1970-03-01,March-1970
1970-04-01,63548350343c9f2921dbb165,1,terepaima,719253,1632.0,1001.651794,986.0,9.96,-69.38,None,None,None,1.815903,738450,1970-04-01,April-1970
1970-05-01,63548350343c9f2921dbb174,1,terepaima,719283,1632.0,1001.651794,986.0,9.96,-69.38,None,None,None,3.538272,738450,1970-05-01,May-1970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01,6354837d343c9f2921dbe2a6,1,terepaima,738156,1632.0,1001.651794,986.0,9.96,-69.38,None,None,None,0.141664,738450,2022-01-01,January-2022
2022-02-01,6354837d343c9f2921dbe2c4,1,terepaima,738187,1632.0,1001.651794,986.0,9.96,-69.38,None,None,None,0.386378,738450,2022-02-01,February-2022
2022-03-01,6354837d343c9f2921dbe2d3,1,terepaima,738215,1632.0,1001.651794,986.0,9.96,-69.38,None,None,None,0.579085,738450,2022-03-01,March-2022


# Estudio Precipitación

In [9]:
pd_precipitacion = data_pandas[['id_point', 'latitud', 'longitud',
                                'precipitacion_mm']]
#pd_precipitacion.head()

# Cargando data SST

In [10]:
# Realizando consulta
data_sst = db.estimateSSTNino34.find()

# Generando pandas dataframe
pd_sst = pd.DataFrame([file for file in data_sst])[['nino34_mean','oni','time']]
pd_sst['periodo'] = pd_sst.time.apply(lambda x: datetime.fromordinal(x))
pd_sst.index = pd.to_datetime(pd_sst.periodo)

In [11]:
# Transformacion
oni_transformacion = MinMaxScaler() #LogMinimax.create( pd_sst.oni.to_numpy() )
oni_transformacion.fit(pd_sst[['oni']])

pd_sst['oni'] = oni_transformacion.transform( pd_sst[['oni']] )

In [12]:
# Transformacion
sst_transformacion = LogMinimax.create( pd_sst.nino34_mean.to_numpy() )

pd_sst['sst_t'] = sst_transformacion.transformacion()

# Integrando base de datos

In [13]:
# Entrenamiento
pd_model = pd.merge(pd_precipitacion.reset_index(drop=False),pd_sst[['oni','sst_t']].reset_index(drop=False),
                    on=['periodo'],
                    how='left'
                    )

# Pronostico
pd_sst_pron = pd_sst[['periodo','oni','sst_t']][pd_sst.periodo > pd_model.periodo.max()].copy()

In [14]:
pd_model

,periodo,id_point,latitud,longitud,precipitacion_mm,oni,sst_t
0,1970-01-01,1,9.96,-69.38,1.698088,0.543835,0.526009
1,1970-02-01,1,9.96,-69.38,0.460838,0.506771,0.549133
2,1970-03-01,1,9.96,-69.38,0.559042,0.497505,0.585536
3,1970-04-01,1,9.96,-69.38,1.815903,0.476123,0.698842
4,1970-05-01,1,9.96,-69.38,3.538272,0.444048,0.676378
...,...,...,...,...,...,...,...
624,2022-01-01,1,9.96,-69.38,0.141664,0.228083,0.280822
625,2022-02-01,1,9.96,-69.38,0.386378,0.234498,0.335206
626,2022-03-01,1,9.96,-69.38,0.579085,0.222381,0.424597
627,2022-04-01,1,9.96,-69.38,4.349987,0.205987,0.500842


# Ajustando modelo NARX

In [15]:
# Transformacion
transformacion = LogMinimax.create( pd_precipitacion.precipitacion_mm.to_numpy() )

pd_model['precip_t'] = transformacion.transformacion()

Modelo según ID point

In [16]:
pd_model_id = pd_model[pd_model.id_point==id_point]
pd_model_id.index = pd.to_datetime(pd_model_id.periodo)
pd_model_id = pd_model_id[[y_output,exogena]]

#pd_model_id.head()

Definiendo estructura de datos

In [17]:
x_data, y_data = split_data(pd_model_id,exog_order,auto_order,exog_delay,prediction_order,exogena,y_output)

Entrenamiento y validación

In [18]:
x_train = x_data[:-prediction_order]
x_vasl = x_data[-prediction_order:]

y_train = y_data[:-prediction_order]
y_vasl = y_data[-prediction_order:]

Modelo NARX

In [19]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [20]:
# Metrícas
mae = keras.metrics.MeanAbsoluteError()
rmse = keras.metrics.RootMeanSquaredError()

In [21]:
confi = {'Input':{'batch_size':None,
                'name':'input',
                'dtype':None,
                'sparse':None,
                'tensor':None,
                'ragged':None,
                'type_spec':None},
        'Dense':{'use_bias':True,
                'kernel_regularizer':None,
                'bias_regularizer':None,
                'activity_regularizer':None,
                'kernel_constraint':None,
                'bias_constraint':None
                }
        }

In [22]:
total = int(2*x_train.shape[-1]/3)
n_neurons = [int(3*total/6), int(2*total/6), int(1*total/6), 1]

activation = len(n_neurons)*[f_activation]
kernel_initializer = 'lecun_normal'
bias_initializer = 'zeros'

In [23]:
# Modelo
model = keras.models.Sequential()

# Entradas
model.add(keras.layers.Input(shape=(x_train.shape[-1],),
                                    batch_size = confi.get('Input').get('batch_size'),
                                    name = confi.get('Input').get('name'),
                                    dtype = confi.get('Input').get('dtype'),
                                    sparse = confi.get('Input').get('sparse'),
                                    tensor = confi.get('Input').get('tensor'),
                                    ragged = confi.get('Input').get('ragged'),
                                    type_spec = confi.get('Input').get('type_spec')
                                    ))

model.add(keras.layers.Dense(   units=n_neurons[0],
                                activation=activation[0],
                                use_bias = confi.get('Dense').get('use_bias'),
                                kernel_initializer=kernel_initializer,
                                bias_initializer=bias_initializer,
                                kernel_regularizer = confi.get('Dense').get('kernel_regularizer'),
                                bias_regularizer = confi.get('Dense').get('bias_regularizer'),
                                activity_regularizer = confi.get('Dense').get('activity_regularizer'),
                                kernel_constraint = confi.get('Dense').get('kernel_constraint'),
                                bias_constraint = confi.get('Dense').get('bias_constraint')
                                ))
                                
model.add(keras.layers.Dropout(0.1))

# Hidden Leyers
if len(n_neurons)>1:
    for index in list( range(1, len(n_neurons)) ):

        model.add(keras.layers.Dense(   units=n_neurons[index],
                                        activation=activation[index],
                                        use_bias = confi.get('Dense').get('use_bias'),
                                        kernel_initializer=kernel_initializer,
                                        bias_initializer=bias_initializer,
                                        kernel_regularizer = confi.get('Dense').get('kernel_regularizer'),
                                        bias_regularizer = confi.get('Dense').get('bias_regularizer'),
                                        activity_regularizer = confi.get('Dense').get('activity_regularizer'),
                                        kernel_constraint = confi.get('Dense').get('kernel_constraint'),
                                        bias_constraint = confi.get('Dense').get('bias_constraint')
                                        ))
                                        
        # model.add(keras.layers.Dropout(0.001))
        # print()

# Out
model.add(keras.layers.Dense(   units=1,
                                activation='linear',
                                kernel_initializer=kernel_initializer,
                                bias_initializer=bias_initializer
                                ))
                                

model.compile(loss='mean_squared_error', optimizer='adam', metrics=[mae,rmse]) 

In [24]:
callback = keras.callbacks.EarlyStopping(
                                            monitor="loss",
                                            min_delta=0,
                                            patience=patience,
                                            verbose=0,
                                            mode="min",
                                            baseline=None,
                                            restore_best_weights=False,
                                        )

Entrenamiento

In [25]:
history = model.fit(x=x_train,
                    y=y_train,
                    epochs=epochs,
                    batch_size=1,
                    verbose=0,
                    workers=2,
                    callbacks=[callback])

In [26]:
print(f'Total epocas:{len(history.epoch)}')

Total epocas:23


Evaluación entrenamiento

In [27]:
# make predictions
trainPredict = model.predict(x_train, verbose=0).reshape(-1)
testPredict = model.predict(x_vasl, verbose=0).reshape(-1)

In [28]:
# Data de test
trainind_pd = pd.DataFrame(trainPredict,
                            index = pd_model_id[-(trainPredict.shape[0]+prediction_order):-(prediction_order)].index,
                            columns=['prediction']
                            )

trainind_pd[y_output] = y_train.reshape(-1)
trainind_pd['type'] = 'training'

trainind_pd['precipitacion_mm'] = trainind_pd[y_output].apply(lambda x: transformacion.inversa(x) if np.isnan(x)==False else np.nan )
trainind_pd['prediction_precipitacion_mm'] = trainind_pd['prediction'].apply(lambda x: transformacion.inversa(x) if np.isnan(x)==False else np.nan )

trainind_pd = pd.merge(trainind_pd,pd_model_id[[exogena]].reset_index(drop=False),
                        on=['periodo'],
                        how='left')

trainind_pd.index = pd.to_datetime(trainind_pd.periodo)

# Validacion entrenamiento
trainig_metrics = metrics(observado=trainind_pd.precipitacion_mm,
                          prediccion=trainind_pd.prediction_precipitacion_mm)

Evaluación validación

In [29]:
# Data de Validacion
validation_pd = pd.DataFrame(testPredict,
                            index = pd_model_id[-prediction_order:].index,
                            columns=['prediction']
                            )

validation_pd[y_output] = y_vasl.reshape(-1)
validation_pd['type'] = 'validation'

validation_pd['precipitacion_mm'] = validation_pd[y_output].apply(lambda x: transformacion.inversa(x) if np.isnan(x)==False else np.nan )
validation_pd['prediction_precipitacion_mm'] = validation_pd['prediction'].apply(lambda x: transformacion.inversa(x) if np.isnan(x)==False else np.nan )

validation_pd = pd.merge(validation_pd,pd_model_id[[exogena]].reset_index(drop=False),
                        on=['periodo'],
                        how='left')

# Validacion entrenamiento
validation_metrics = metrics(observado=validation_pd.precipitacion_mm,
                          prediccion=validation_pd.prediction_precipitacion_mm)

Test

In [30]:
data_exogena = pd_model_id[-prediction_order:][[exogena]]
data_exogena[y_output] = np.nan
data_predict = pd_model_id[pd_model_id.index < data_exogena.index.min()][[y_output,exogena]]

In [31]:
pd_test = predict_one_stap_narx(model,data_predict,data_exogena,exog_order,auto_order,exog_delay,prediction_order,exogena,y_output)

pd_test = pd_test.rename(columns={y_output:'prediction'})
pd_test['type'] = 'test'

pd_test['precip_t'] = pd_model_id[pd_model_id.index > trainind_pd.periodo.max()][y_output]

pd_test['precipitacion_mm'] = pd_test[y_output].apply(lambda x: transformacion.inversa(x) if np.isnan(x)==False else np.nan )
pd_test['prediction_precipitacion_mm'] = pd_test['prediction'].apply(lambda x: transformacion.inversa(x) if np.isnan(x)==False else np.nan )

# Validacion entrenamiento
test_metrics = metrics(observado=pd_test.precipitacion_mm,
                        prediccion=pd_test.prediction_precipitacion_mm)
test_metrics

{'mape': 189.01612160724622,
 'mae': 1.0871622331000894,
 'mse': 1.2811772231641188,
 'rmse': 1.6414150771545222,
 'r2': -0.0010289286103450923}

Resultados del modelo

In [32]:
# Resultados del modelo
dict_metrics = {'epocas':[len(history.epoch)],
                'prediction_order':[prediction_order],
                'auto_order':[auto_order],
                'exog_order':[exog_order],
                'exog_delay':[exog_delay],
                'activation':[activation[0]],
                'id_point':[id_point],
                'n_neurons':str(n_neurons),
                'capas':[len(n_neurons)],
                'training_mse':[trainig_metrics["mse"]],
                'training_rmse':[trainig_metrics["rmse"]],
                'training_mae':[trainig_metrics["mae"]],
                'trainig_mape':[trainig_metrics['mape']],
                'trainig_r':[trainig_metrics['r2']],
                'validation_mse':[validation_metrics["mse"]],
                'validation_rmse':[validation_metrics["rmse"]],
                'validation_mae':[validation_metrics["mae"]],
                'validation_mape':[validation_metrics['mape']],
                'validation_r':[validation_metrics['r2']],
                'test_mse':[test_metrics["mse"]],
                'test_rmse':[test_metrics["rmse"]],
                'test_mae':[test_metrics["mae"]],
                'test_mape':[test_metrics['mape']],
                'test_r':[test_metrics['r2']]
                }

experimento_pd = pd.DataFrame.from_dict(dict_metrics)

Pronóstico

In [33]:
data_predict = pd_model_id[[y_output,exogena]]

data_exogena = pd_sst_pron[pd_sst_pron.index>data_predict.index.max()][[exogena]]
data_exogena[y_output] = np.nan
data_exogena = data_exogena[[y_output,exogena]]

pd_prediction = predict_one_stap_narx(model,data_predict,data_exogena,exog_order,auto_order,exog_delay,prediction_order, exogena, y_output)

pd_prediction = pd_prediction.rename(columns={y_output:'prediction'})
pd_prediction['type'] = 'prediction'

pd_prediction['precipitacion_mm'] = np.nan
pd_prediction['prediction_precipitacion_mm'] = pd_prediction['prediction'].apply(lambda x: transformacion.inversa(x) if np.isnan(x)==False else np.nan )

pd_prediction['precip_t'] = np.nan

In [34]:
columns = list(pd_prediction.reset_index(drop=False))
print(columns)
# Uniendo informacion
pd_summary = pd.concat([trainind_pd.reset_index(drop=True)[columns], 
                        pd_test.reset_index(drop=False)[columns], 
                        validation_pd[columns], 
                        pd_prediction.reset_index(drop=False)[columns]
                        ])

['periodo', 'prediction', 'oni', 'type', 'precipitacion_mm', 'prediction_precipitacion_mm', 'precip_t']


In [35]:
pd_summary.type.unique()

array(['training', 'test', 'validation', 'prediction'], dtype=object)

Logica de guardado

In [36]:
model_confi = {"id_point":id_point,
            "n_neurons":n_neurons,
            "activation":activation,
            "prediction_order":prediction_order,
            "auto_order":auto_order,
            "exog_order":exog_order,
            "exog_delay":exog_delay,
            "metrics":dict_metrics
            }

In [37]:
import pickle

if os.listdir(f'{DIR}{experimento}') == []:

    # Modelo
    model.save(f'{DIR}{experimento}/model.h5')

    # Pesos
    model.save_weights(f'{DIR}{experimento}/weights.h5')

    # History
    with open(f'{DIR}{experimento}/history.pkl', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

    # confi
    with open(f'{DIR}{experimento}/model_confi.pkl', 'wb') as file_pi:
        pickle.dump(model_confi, file_pi)
    
    # guardando resultados
    pd_summary.to_pickle(f'{DIR}{experimento}/predicciones.pkl')

else:
    files = [x for x in os.listdir(f'{DIR}{experimento}') if x.find('summary')!=-1 ]
    total_summary = pd.concat([pd.read_csv(f'{DIR}{experimento}/{file}') for file in files])
    print( f"Actual: {validation_metrics['r2']}; Best Model: {total_summary.validation_r.max()}" )

    if validation_metrics['r2'] > total_summary.validation_r.max(): 

        # Modelo
        model.save(f'{DIR}{experimento}/model.h5')

        # Pesos
        model.save_weights(f'{DIR}{experimento}/weights.h5')

        # History
        with open(f'{DIR}{experimento}/history.pkl', 'wb') as file_pi:
            pickle.dump(history.history, file_pi)
        
        # confi
        with open(f'{DIR}{experimento}/model_confi.pkl', 'wb') as file_pi:
            pickle.dump(model_confi, file_pi)
        
        # guardando resultados
        pd_summary.to_pickle(f'{DIR}{experimento}/predicciones.pkl')

Guardando Summary

In [38]:
experi = f'{DIR}{experimento}/{id_point}_{len(n_neurons)}_{activation[0]}_{prediction_order}_{auto_order}_{exog_order}_{exog_delay}'
experimento_pd.to_csv(f'{experi}_summary.csv',index=False)